In [1]:


import pandas as pd

df = pd.read_csv('../reverse_image_search.csv')
df.head()



,id,path,label
0,0,./train/brain_coral/n01917289_1783.JPEG,brain_coral
1,1,./train/brain_coral/n01917289_4317.JPEG,brain_coral
2,2,./train/brain_coral/n01917289_765.JPEG,brain_coral
3,3,./train/brain_coral/n01917289_1079.JPEG,brain_coral
4,4,./train/brain_coral/n01917289_2484.JPEG,brain_coral


In [2]:


import cv2
from towhee.types.image import Image

id_img = df.set_index('id')['path'].to_dict()
def read_images(results):
    imgs = []
    for re in results:
        path = id_img[re.id]
        imgs.append(Image(cv2.imread(path), 'BGR'))
    return imgs



/home/kasim/anaconda3/envs/milvus/lib/python3.12/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [3]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

def create_milvus_collection(collection_name, dim):
    connections.connect(host='127.0.0.1', port='19530')
    
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)
    
    fields = [
    FieldSchema(name='path', dtype=DataType.VARCHAR, description='path to image', max_length=500, 
                    is_primary=True, auto_id=False),
    FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, description='embedding vectors', dim=dim)
    ]
    schema = CollectionSchema(fields=fields, description='text image search')
    collection = Collection(name=collection_name, schema=schema)

    # create IVF_FLAT index for collection.
    index_params = {
        'metric_type':'L2',
        'index_type':"IVF_FLAT",
        'params':{"nlist":512}
    }
    collection.create_index(field_name="embedding", index_params=index_params)
    return collection

#collection = create_milvus_collection('text_image_search', 512)

In [4]:
from pymilvus import Collection, utility
connections.connect(host='127.0.0.1', port='19530')

collection = create_milvus_collection('text_image_search', 512)

In [5]:


from towhee import ops, pipe, DataCollection
import numpy as np



In [6]:


p = (
    pipe.input('path')
    .map('path', 'img', ops.image_decode.cv2('rgb'))
    .map('img', 'vec', ops.image_text_embedding.clip(model_name='clip_vit_base_patch16', modality='image'))
    .map('vec', 'vec', lambda x: x / np.linalg.norm(x))
    .output('img', 'vec')
)

DataCollection(p('./teddy.png')).show()



/home/kasim/anaconda3/envs/milvus/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[ WARN:0@24.198] global loadsave.cpp:248 findDecoder imread_('./teddy.png'): can't open/read file: check file path/integrity
2024-03-31 13:12:07,680 - 126739130873408 - image_decode_cv2.py-image_decode_cv2:68 - ERROR: Read image ./teddy.png failed


RuntimeError: Node-image-decode/cv2-0 runs failed, error msg: Read image ./teddy.png failed, Traceback (most recent call last):
  File "/home/kasim/anaconda3/envs/milvus/lib/python3.12/site-packages/towhee/runtime/nodes/node.py", line 158, in _call
    return True, self._op(*inputs), None
                 ^^^^^^^^^^^^^^^^^
  File "/home/kasim/.towhee/operators/image-decode/cv2/versions/main/image_decode_cv2.py", line 69, in __call__
    raise RuntimeError(err)
RuntimeError: Read image ./teddy.png failed
, Traceback (most recent call last):
  File "/home/kasim/anaconda3/envs/milvus/lib/python3.12/site-packages/towhee/runtime/nodes/node.py", line 171, in process
    self.process_step()
  File "/home/kasim/anaconda3/envs/milvus/lib/python3.12/site-packages/towhee/runtime/nodes/_map.py", line 63, in process_step
    assert succ, msg
AssertionError: Read image ./teddy.png failed, Traceback (most recent call last):
  File "/home/kasim/anaconda3/envs/milvus/lib/python3.12/site-packages/towhee

In [8]:


p2 = (
    pipe.input('text')
    .map('text', 'vec', ops.image_text_embedding.clip(model_name='clip_vit_base_patch16', modality='text'))
    .map('vec', 'vec', lambda x: x / np.linalg.norm(x))
    .output('text', 'vec')
)

DataCollection(p2("cat")).show()



text,vec
cat,"[0.033149168, -0.020378174, 0.000116232535, ...] shape=(512,)"


In [9]:
p = DataCollection(p2("A teddybear on a skateboard in Times Square."))


In [10]:
%%time
collection = create_milvus_collection('text_image_search', 512)

def read_csv(csv_path, encoding='utf-8-sig'):
    import csv
    with open(csv_path, 'r', encoding=encoding) as f:
        data = csv.DictReader(f)
        for line in data:
            yield int(line['id']), line['path']

p3 = (
    pipe.input('csv_file')
    .flat_map('csv_file', ('id', 'path'), read_csv)
    .map('path', 'img', ops.image_decode.cv2('rgb'))
    .map('img', 'vec', ops.image_text_embedding.clip(model_name='clip_vit_base_patch16', modality='image', device=0))
    .map('vec', 'vec', lambda x: x / np.linalg.norm(x))
    .map(('path', 'vec'), (), ops.ann_insert.milvus_client(host='127.0.0.1', port='19530', collection_name='text_image_search'))
    .output()
)

ret = p3('reverse_image_search.csv')

RuntimeError: Node-image-text-embedding/clip-2 runs failed, error msg: Create image-text-embedding/clip-2 operator image-text-embedding/clip:main with args None and kws {'model_name': 'clip_vit_base_patch16', 'modality': 'image', 'device': 0} failed, err: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx, Traceback (most recent call last):
  File "/home/kasim/anaconda3/envs/milvus/lib/python3.12/site-packages/towhee/runtime/nodes/node.py", line 88, in initialize
    self._op = self._op_pool.acquire_op(
               ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/kasim/anaconda3/envs/milvus/lib/python3.12/site-packages/towhee/runtime/operator_manager/operator_pool.py", line 106, in acquire_op
    op = self._op_loader.load_operator(hub_op_id, op_args, op_kws, tag, latest)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/kasim/anaconda3/envs/milvus/lib